# Домашнее задание 3
## Text Normalization 

deadline: 15 мая 2020, 23:30

В этом домашнем задании вы будете работать с корпусом соревнования по нормализации текстов на русском языке. 

Ссылка на соревнование:
https://www.kaggle.com/c/text-normalization-challenge-russian-language

Корпус (train-test split) доступен там же, на kaggle. Кроме того, kaggle проверяет результаты на тестовом множестве. Пример сабмита в файле: ru_sample_submission_2. 

Задача заключается в том, привести исходный текст (колонку before) в нормализованную форму (колонка after). Дополнительно известны классы токенов (колонка class), общее число классов – 15. В тестовом множестве классы токенов отсутствуют. 

Корпус состоит из предложений на русском языке и их нормализованных аналогов. Примеры продемонстрированы на kaggle.

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 3-х человек.
2. Домашнее задание сдается через anytask, инвайты будут дополнительно высланы.
3. Домашнее задание оформляется в виде отчета либо в .pdf файле, либо ipython-тетрадке. 
4. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
5. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
6. Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
7. Плагиат и любое недобросоветсное цитирование приводит к обнуление оценки. 

In [2]:
# !pip install kaggle

In [3]:
os.environ['KAGGLE_USERNAME'] = 'darlitlit'
os.environ['KAGGLE_KEY'] = 'dcc30c94bda5fb17c97cc9d0f6c8e407'

In [15]:
!kaggle competitions download -c text-normalization-challenge-russian-language

 93%|█████████████████████████████████████   | 111M/120M [00:02<00:00, 39.5MB/s]
100%|████████████████████████████████████████| 120M/120M [00:02<00:00, 56.9MB/s]


In [17]:
# !unzip text-normalization-challenge-russian-language.zip

## Часть 1. [1 балл] Эксплоративный анализ

1. Найдите примеры каждого класса и опишите, по какой логике проведена нормализация токенов разных классов. 
2. В каких случаях токены класса PLAIN подвергаются нормализации? 
3. Напишите правила для нормализации токенов класса ORDINAL. 

## Часть 2. [6 баллов]  seq2seq архитектуры
Имплементируйте несколько seq2seq архитектур. Энкодер получает на вход последовательность токенов before, декодер учится превращать их в токены after.
Энкодер и декодер работают на уровне символов, эмбеддинги символов инициализируются случайно (по аналогии с работами, в которых предложены нейросетевые модели исправления опечаток).

Эту часть задания рекомендуется выполнять с использованием allennlp (должно быть проще и удобнее).

1. [3 балла] LSTM encoder + LSTM decoder + три механизма внимания: скалярное произведение, аддитивное внимание и мультипликативное внимание (см. лекцию 6, слайд "подсчет весов attention")
2. [3 балла] Transformer

Используя автопровереку kaggle, оцените, как влияют параметры архитектуры на качество задачи.

[бонус] convolutional encoder + convolutional decoder

[бонус] pyramid LSTM (размер l+1 слоя в два раз меньше размера l, i-тый вход l+1 слоя – конкатенация выходов 2i и 2i+1)

## Часть 3. [2 балла]  Дополнительные признаки
Предложите и покажите, как можно было бы повысить качество нейросетевых моделей. Примерные варианты:
1. ансамблирование нейронных сетей
2. добавление морфологоческих признаков 
3. использование эмбеддингов слов 


## Часть 4. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Проведите анализ ошибок: когда модель ошибается? Можно ли скзаать, почему модель ошибается? Сравните результаты всех разработанных моделей. Что помогло вам в выполнении работы, чего не хватало?

# Часть 1. Эксплоративный анализ

1. Найдите примеры каждого класса и опишите, по какой логике проведена нормализация токенов разных классов. 
2. В каких случаях токены класса PLAIN подвергаются нормализации? 
3. Напишите правила для нормализации токенов класса ORDINAL. 

In [29]:
# !pip install tabulate

In [1]:
import os
import numpy as np

import pandas as pd
from tabulate import tabulate

In [31]:
train_df = pd.read_csv('ru_train.csv.zip')

In [32]:
train_df.head()

,sentence_id,token_id,class,before,after
0,0,0,PLAIN,По,По
1,0,1,PLAIN,состоянию,состоянию
2,0,2,PLAIN,на,на
3,0,3,DATE,1862 год,тысяча восемьсот шестьдесят второй год
4,0,4,PUNCT,.,.


In [52]:
test_df = pd.read_csv('ru_test.csv.zip')

In [53]:
test_df.head()

,sentence_id,token_id,before
0,0,0,Производится
1,0,1,в
2,0,2,Азии
3,0,3,","
4,0,4,Африке


In [56]:
examples = pd.read_csv('ru_sample_submission_2.csv.zip')

In [57]:
examples.head()

,id,after
0,0_0,Эта
1,0_1,книга
2,0_2,","
3,0_3,отличающаяся
4,0_4,«


## Найдите примеры каждого класса и опишите, по какой логике проведена нормализация токенов разных классов.

In [7]:
CLS = list(set(train_df['class'].values))

In [51]:
n_examples = 10
for cls in CLS:
    idx = train_df['class'] == cls
    examples = train_df[idx].sample(n_examples)
    print(tabulate(examples[['before', 'after', 'class']],
                   headers=['before', 'after', 'class'],
                   showindex=False))

before    after    class
--------  -------  -------
)         )        PUNCT
,         ,        PUNCT
)         )        PUNCT
.         .        PUNCT
)         )        PUNCT
)         )        PUNCT
«         «        PUNCT
.         .        PUNCT
,         ,        PUNCT
.         .        PUNCT
before    after    class
--------  -------  --------
-         -        VERBATIM
-         -        VERBATIM
Β         бета     VERBATIM
-         -        VERBATIM
-         -        VERBATIM
-         -        VERBATIM
-         -        VERBATIM
-         -        VERBATIM
店        店       VERBATIM
ր         ր        VERBATIM
  before  after                    class
--------  -----------------------  -------
    0185  ноль один восемь пять    DIGIT
      06  ноль шесть               DIGIT
    0941  ноль девять четыре один  DIGIT
      07  ноль семь                DIGIT
      06  ноль шесть               DIGIT
     000  ноль ноль ноль           DIGIT
    0341  ноль три четыре один     DI

before            after                                                                                                        class
----------------  -----------------------------------------------------------------------------------------------------------  ----------
vishime.ru        в_trans и_trans ш_trans и_trans м_trans точка р_trans у_trans                                                ELECTRONIC
tolweb.org        т_trans о_trans л_trans у_trans э_trans б_trans точка о_trans р_trans г_trans                                ELECTRONIC
Shakira.com       ш_trans е_trans й_trans к_trans и_trans р_trans а_trans точка к_trans о_trans м_trans                        ELECTRONIC
OM.net            о_trans м_trans точка н_trans е_trans т_trans                                                                ELECTRONIC
Sportal.bg        с_trans п_trans о_trans р_trans т_trans а_trans л_trans точка b g                                            ELECTRONIC
www.enric.es      w w w точка э_trans н

'PUNCT' -- знаки пунктуация, никак не меняются

'VERBATIM' -- как вслух произносится символ ("в квадрате", "нижнее подчеркивание", буквы греческого алфавита)

'DIGIT' -- числа

'MEASURE' -- как вслух произносятся сокращения для единиц измерения

'TELEPHONE' -- телефон, вместо - ставится sil

'LETTERS' -- пробелом разделяются аббревиатуры

'PLAIN' -- слова, написанные на русском, не меняются, иначе -- транскрибируются

'DECIMAL' -- числа, дробные и с сокращениями (тыс, млн)

'ORDINAL' -- порядковые числительные

'FRACTION' -- дроби

'ELECTRONIC' -- как произносятся ссылки (транскрипция)

'DATE' -- даты

'MONEY' -- числа и валютные названия (транскрипцией, сша -- сэ ш а)

'TIME' -- время

'CARDINAL' -- числа в именительном падеже

## В каких случаях токены класса PLAIN подвергаются нормализации?

In [52]:
plain = train_df[train_df['class'] == 'PLAIN']
plain[plain.before != plain.after].head()

,sentence_id,token_id,class,before,after
43,3,7,PLAIN,Tiberius,т_trans и_trans б_trans е_trans р_trans и_tran...
44,3,8,PLAIN,Julius,д_trans ж_trans у_trans л_trans и_trans у_tran...
45,3,9,PLAIN,Pollienus,п_trans о_trans л_trans л_trans и_trans е_tran...
46,3,10,PLAIN,Auspex,о_trans с_trans п_trans е_trans к_trans с_trans
153,10,4,PLAIN,Half,х_trans а_trans л_trans ф_trans


'PLAIN' нормализуются (транслитерируются), если написаны нерусскими буквами

## Напишите правила для нормализации токенов класса ORDINAL

In [64]:
ordinal = train_df[train_df['class'] == 'ORDINAL']
ordinal.head()

,sentence_id,token_id,class,before,after
53,3,17,ORDINAL,III,третьего
163,11,1,ORDINAL,1895,тысяча восемьсот девяносто пятом
164,11,2,ORDINAL,—1896,тысяча восемьсот девяносто шестом
485,35,5,ORDINAL,1796,тысяча семьсот девяносто шестого
487,35,7,ORDINAL,1802,тысяча восемьсот второй


In [65]:
with_ordinals = list(set(ordinal['sentence_id'].values))
with_ordinals = train_df[train_df['sentence_id'].isin(with_ordinals)]

In [74]:
before_df = with_ordinals.groupby(['sentence_id'])['before'].apply(list)
after_df = with_ordinals.groupby(['sentence_id'])['after'].apply(list)

In [76]:
before_df = pd.DataFrame({'text': before_df}).reset_index()
before_df = before_df.text.apply(lambda x: ' '.join(x))

In [84]:
after_df = pd.DataFrame({'text': after_df}).reset_index()
after_df = after_df.text.apply(lambda x: ' '.join(x))

In [87]:
before_df

0        Тиберий Юлий Поллиен Ауспекс ( лат . Tiberius ...
1        в 1895 —1896 годах служил на Черноморском флот...
2        После Павловских реформ , с 1796 по 1802 год ,...
3        Некоторые источники указывают на период 1109 —...
4        Джон Стюарт , 1-й маркиз Бьют , стал носить эт...
                               ...                        
33236    Сформирована 21 апреля 1944 года путем преобра...
33237    Соответственно отодвигался и ввод в прорыв 6-г...
33238    В первом тайме на 7-й минуте Зинедин Зидан реа...
33239    Packard Station Sedan — это легковой автомобил...
33240    В отличие от зенитных артиллерийских полков в ...
Name: text, Length: 33241, dtype: object

## Часть 2. [6 баллов]  seq2seq архитектуры
Имплементируйте несколько seq2seq архитектур. Энкодер получает на вход последовательность токенов before, декодер учится превращать их в токены after.
Энкодер и декодер работают на уровне символов, эмбеддинги символов инициализируются случайно (по аналогии с работами, в которых предложены нейросетевые модели исправления опечаток).

Эту часть задания рекомендуется выполнять с использованием allennlp (должно быть проще и удобнее).

1. [3 балла] LSTM encoder + LSTM decoder + три механизма внимания: скалярное произведение, аддитивное внимание и мультипликативное внимание (см. лекцию 6, слайд "подсчет весов attention")
2. [3 балла] Transformer

Используя автопровереку kaggle, оцените, как влияют параметры архитектуры на качество задачи.

[бонус] convolutional encoder + convolutional decoder

[бонус] pyramid LSTM (размер l+1 слоя в два раз меньше размера l, i-тый вход l+1 слоя – конкатенация выходов 2i и 2i+1)

### LSTM encoder + LSTM decoder + три механизма внимания: скалярное произведение, аддитивное внимание и мультипликативное внимание (см. лекцию 6, слайд "подсчет весов attention")

Скалярное произведение -- allennlp.modules.attention.dot_product_attention.DotProductAttention


Аддитивное внимание -- allennlp.modules.attention.additive_attention.AdditiveAttention


Мультипликативное внимание -- allennlp.modules.attention.bilinear_attention.BilinearAttention

In [10]:
# !pip install allennlp

In [145]:
import torch

from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.modules.attention import LinearAttention
from allennlp.nn.activations import Activation
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.token_indexers import TokenCharactersIndexer
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data import Instance

from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
import torch.optim as optim
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer

In [226]:
# sentence_df = train_df.groupby(['sentence_id'])['after'].apply(list)

In [227]:
# sentence_df.head()
# 

In [228]:
# sentence_df = pd.DataFrame({'text': sentence_df}).reset_index()

In [229]:
# sentence_df.text = sentence_df.text.apply(lambda x: ' '.join(map(str, x)))

In [230]:
# sentence_df.to_csv('after_train.csv')

In [231]:
len(sentence_df)

761436

In [323]:
class MyDataReader(DatasetReader):
    def __init__(self, tokenizer, source_token_indexers, target_token_indexers, max_rows=None):
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.source_token_indexers = source_token_indexers
        self.target_token_indexers = target_token_indexers
        self.max_rows = max_rows

    def text_to_instance(self, input_tokens, target_tokens):
        input_field = TextField(input_tokens, self.source_token_indexers)
        target_field = TextField(target_tokens, self.target_token_indexers)

        fields = {"source_tokens": input_field, "target_tokens": target_field}

        return Instance(fields)
    
    def _get_sentences(self, data, column):
        sentence_df = data.groupby(['sentence_id'])[column].apply(list)
        sentence_df = pd.DataFrame({'text': sentence_df}).reset_index()
        sentence_df.text = sentence_df.text.apply(lambda x: ' '.join(map(str, x)))
        
        return sentence_df
    
    def _read(self, before_path):
        before = pd.read_csv('before_train.csv')
        after = pd.read_csv('after_train.csv')
        
        i = 0
        for text, norm_text in zip(before.text.values, after.text.values):
            before_tokens = self.tokenizer.tokenize(text)
            after_tokens = self.tokenizer.tokenize(norm_text)
            yield self.text_to_instance(before_tokens, after_tokens)
            i += 1
            if self.max_rows and i >= self.max_rows:
                break

In [324]:
# source_token_indexer =  {
#     'tokens': 
#     TokenCharactersIndexer(character_tokenizer=tokenizer, min_padding_length=1, namespace='tokens')
# }
# target_token_indexer =  {
#     'tokens':
#     TokenCharactersIndexer(character_tokenizer=tokenizer, min_padding_length=1, namespace='target_tokens')
# }

In [325]:
source_token_indexer =  {
    'tokens': 
    SingleIdTokenIndexer(namespace='tokens')
}
target_token_indexer =  {
    'tokens':
    SingleIdTokenIndexer(namespace='target_tokens')
}

In [326]:
tokenizer = CharacterTokenizer(byte_encoding=None, lowercase_characters=True,
                               start_tokens=['<sos>'], end_tokens=['<eos>'])

In [329]:
dr = MyDataReader(tokenizer, source_token_indexer, target_token_indexer)
data_set = dr.read('before_train.csv')

761436it [04:49, 2630.38it/s]


In [331]:
small_data_set = data_set[:1000]

In [328]:
print(data_set[0])

Instance with fields:
 	 source_tokens: TextField of length 28 with text: 
 		[<sos>, п, о,  , с, о, с, т, о, я, н, и, ю,  , н, а,  , 1, 8, 6, 2,  , г, о, д,  , ., <eos>]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 target_tokens: TextField of length 58 with text: 
 		[<sos>, п, о,  , с, о, с, т, о, я, н, и, ю,  , н, а,  , т, ы, с, я, ч, а,  , в, о, с, е, м, ь, с, о,
		т,  , ш, е, с, т, ь, д, е, с, я, т,  , в, т, о, р, о, й,  , г, о, д,  , ., <eos>]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 



In [299]:
data_set[0]['source_tokens']._token_indexers

{'tokens': <allennlp.data.token_indexers.single_id_token_indexer.SingleIdTokenIndexer at 0x7fa85d119290>}

In [300]:
# from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader

In [217]:
reader = Seq2SeqDatasetReader(
        source_tokenizer=tokenizer,
        target_tokenizer=tokenizer,
        source_token_indexers={'tokens': SingleIdTokenIndexer()},
        target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')})
_dataset = reader.read('before_train.tsv')

761437it [03:36, 3516.34it/s] 


In [280]:
print(_dataset[1])

Instance with fields:
 	 source_tokens: TextField of length 5 with text: 
 		[@start@, <sos>, 0, <eos>, @end@]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 target_tokens: TextField of length 30 with text: 
 		[@start@, <sos>, п, о,  , с, о, с, т, о, я, н, и, ю,  , н, а,  , 1, 8, 6, 2,  , г, о, д,  , .,
		<eos>, @end@]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 



In [247]:
_dataset[0]['source_tokens']._token_indexers

{'tokens': <allennlp.data.token_indexers.single_id_token_indexer.SingleIdTokenIndexer at 0x7f9951480610>}

In [185]:
# bf = pd.read_csv('after_train.csv')

In [186]:
# bf.columns

Index(['Unnamed: 0', 'sentence_id', 'text'], dtype='object')

In [187]:
# bf.drop(columns=['Unnamed: 0', 'sentence_id']).to_csv('after_train.tsv', sep='\t')

In [184]:
# all_dataset = reader.read('before_train.tsv')

761437it [02:36, 4876.07it/s] 


In [188]:
# all_dataset_y = reader.read('after_train.tsv')

761437it [03:46, 3365.67it/s]


In [129]:
# data_set[0]['source_tokens']

In [332]:
vocab = Vocabulary.from_instances(data_set,
                                  min_count={'source_tokens': 1,
                                             'target_tokens': 1})

100%|██████████| 761436/761436 [01:04<00:00, 11725.76it/s]


In [302]:
vocab

Vocabulary with namespaces:  tokens, Size: 131 || target_tokens, Size: 123 || Non Padded Namespaces: {'*labels', '*tags'}

In [333]:
EN_EMBEDDING_DIM = 256
HIDDEN_DIM = 20

In [334]:
encoder = PytorchSeq2SeqWrapper(
    torch.nn.LSTM(EN_EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

In [335]:
attention = LinearAttention(HIDDEN_DIM, HIDDEN_DIM, activation=Activation.by_name('tanh')())

In [336]:
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EN_EMBEDDING_DIM)

word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

In [337]:
max_decoding_steps = 10
ZH_EMBEDDING_DIM = 256

In [348]:
test_size = len(data_set)*0.1

In [349]:
test_size

76143.6

In [351]:
train_set, val_set = torch.utils.data.random_split(data_set, [len(data_set) - int(test_size),
                                                              int(test_size)], )

In [356]:
train_set.dataset = train_set.dataset[:100000]

In [357]:
model = SimpleSeq2Seq(vocab, word_embeddings, encoder, max_decoding_steps,
                      target_embedding_dim=ZH_EMBEDDING_DIM,
                      target_namespace='target_tokens',
                      beam_size=8,
                      use_bleu=True)

In [362]:
optimizer = optim.Adam(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=256, sorting_keys=[("source_tokens", 'num_tokens')])
iterator.index_with(vocab)

In [ ]:
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_set,
                  validation_dataset=val_set,
                  num_epochs=1)
trainer.train()
# for i in range(50):
#     print('Epoch: {}'.format(i))
    

#     predictor = SimpleSeq2SeqPredictor(model, reader)

#     for instance in itertools.islice(val_set, 10):
#         print('SOURCE:', instance.fields['source_tokens'].tokens)
#         print('GOLD:', instance.fields['target_tokens'].tokens)
#         print('PRED:', predictor.predict_instance(instance)['predicted_tokens'])

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ulyanin/jupyter-notebook/venv/lib/python3.7/site-packages/allennlp/common/util.py", line 378, in gpu_memory_mb
    encoding='utf-8')
  File "/usr/lib/python3.7/subprocess.py", line 411, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.7/subprocess.py", line 488, in run
    with Popen(*popenargs, **kwargs) as process:
  File "/usr/lib/python3.7/subprocess.py", line 800, in __init__
    restore_signals, start_new_session)
  File "/usr/lib/python3.7/subprocess.py", line 1482, in _execute_child
    restore_signals, start_new_session, preexec_fn)
OSError: [Errno 12] Cannot allocate memory
loss: 3.6297 ||: 100%|██████████| 1/1 [00:02<00:00,  2.30s/it]
  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
trainer